<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=72df855adae55b451d32ab8023417817b8d3de603b051200e5448e83b31a782b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-09 14:22:18
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.41 C
-------------------
Today PnL: -1.63 L (-1.14%)
Current PnL: -28.01 L (-17.86%)
CY Booked + Current PnL: -13.72 L (-8.75%)
-------------------
Total profit:  1.66 L
Total loss:  -29.67 L
-------------------
Total Booked + Current PnL: 13.26 L (10.22%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.13%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.46 L (66.25%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.52%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.91,14.17,6.33,21.40,19182.0,37604.0,303030.0,5.95,58.0,X-LC,5.69,14.0,1.96,2.15,21.39,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,0.15,-21.28,27.16,0.09,21545.0,-21450.0,79327.0,84.20,36.0,M-SC,3.31,253.0,-1.00,0.56,4.22,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-1.61,2.89,15.88,19.23,24584.0,4348.0,154808.0,21.35,65.0,L-LC,7.04,182.0,0.18,1.10,24.78,XY25,ATH,MINING
39,INDIGOPNTS,1408.00,-1.54,-14.60,17.12,0.02,25521.0,-25486.0,149073.0,127.12,53.0,M-SC,8.06,240.0,-1.00,1.06,29.67,OX40N,NTT,PAINTS
49,MASFIN,398.61,-3.36,-4.94,28.40,22.05,26450.0,-4845.0,93135.0,-18.25,44.0,H-SC,6.44,168.0,-0.18,0.66,35.38,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,2.26,-17.32,51.80,25.51,79705.0,-32225.0,153870.0,-5.62,46.0,H-LC,1.72,50.0,-0.40,1.09,13.39,AR,ATH,ELECTRICAL
5,ASIANPAINT,3460.25,1.49,-1.16,22.36,20.95,55644.0,-2913.0,248855.0,-2.18,55.0,X-LC,12.25,36.0,-0.05,1.76,34.07,X40,ATH,PAINTS
1,ABB,7934.00,1.37,-6.61,55.87,45.57,136502.0,-17299.0,244320.0,-40.03,45.0,H-MC,4.69,121.0,-0.13,1.73,3.96,AR,NTT,ELECTRICAL
44,JCHAC,2282.00,1.36,-33.96,51.40,-0.01,43386.0,-43396.0,84409.0,17182.61,52.0,M-SC,13.73,235.0,-1.00,0.60,8.03,OX40N,BTT,AC
32,HCLTECH,1922.01,0.84,7.77,15.74,24.73,41036.0,18790.0,260714.0,12.34,56.0,X-LC,6.24,13.0,0.46,1.85,24.19,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,-7.46,-5.73,57.67,48.63,108700.0,-11467.0,188487.0,-37.19,48.0,H-SC,13.73,137.0,-0.11,1.34,5.22,XR,NTT,MISC
56,RAJOOENG,143.10,-5.54,-33.75,110.72,39.61,75190.0,-34590.0,67910.0,-55.26,49.0,H-SC,30.29,136.0,-0.46,0.48,9.92,AR,ATH,MISC
18,COFFEEDAY,80.00,-4.15,-43.41,139.02,35.27,89337.0,-49287.0,64262.0,-55.73,40.0,L-SC,4.18,268.0,-0.55,0.46,57.28,XR,NTT,HOTELS
76,TRIDENT,48.00,-3.98,-30.31,84.40,28.51,54264.0,-27960.0,64294.0,-17.44,37.0,M-SC,7.45,226.0,-0.52,0.46,10.02,XR,NTT,TEXTILES
4,ALKYLAMINE,4546.37,-3.95,-34.24,194.45,93.63,129099.0,-34571.0,66392.0,-29.56,39.0,H-SC,16.68,143.0,-0.27,0.47,0.37,SR,ATH,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.24,-2.98,114.29,107.91,163722.0,-4393.0,143251.0,-20.96,49.0,M-SC,10.13,220.0,-0.03,1.02,6.81,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.54,-14.60,17.12,0.02,25521.0,-25486.0,149073.0,127.12,53.0,M-SC,8.06,240.0,-1.00,1.06,29.67,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.91,-18.36,61.73,32.03,66438.0,-24209.0,107626.0,-11.67,63.0,H-SC,6.89,157.0,-0.36,0.76,20.55,OX40N,NTT,IT
77,TTKPRESTIG,770.00,0.15,-21.28,27.16,0.09,21545.0,-21450.0,79327.0,84.20,36.0,M-SC,3.31,253.0,-1.00,0.56,4.22,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.34,-21.97,45.43,13.47,95590.0,-59256.0,210411.0,-67.92,49.0,H-SC,4.58,159.0,-0.62,1.49,6.99,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-1.61,2.89,15.88,19.23,24584.0,4348.0,154808.0,21.35,65.0,L-LC,7.04,182.0,0.18,1.10,24.78,XY25,ATH,MINING
84,WIPRO,420.00,-0.19,7.34,60.49,72.27,110020.0,12434.0,181882.0,-7.89,49.0,M-LC,6.16,101.0,0.11,1.29,13.97,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.19,7.34,60.49,72.27,110020.0,12434.0,181882.0,-7.89,49.0,M-LC,6.16,101.0,0.11,1.29,13.97,XR,NTT,IT
25,FINCABLES,1641.55,-1.24,-2.98,114.29,107.91,163722.0,-4393.0,143251.0,-20.96,49.0,M-SC,10.13,220.0,-0.03,1.02,6.81,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.11,-7.91,124.49,106.72,141388.0,-9762.0,113574.0,-54.44,34.0,H-SC,7.40,138.0,-0.07,0.81,13.61,AR,ATH,MISC
37,IEX,219.00,-7.46,-5.73,57.67,48.63,108700.0,-11467.0,188487.0,-37.19,48.0,H-SC,13.73,137.0,-0.11,1.34,5.22,XR,NTT,MISC
1,ABB,7934.00,1.37,-6.61,55.87,45.57,136502.0,-17299.0,244320.0,-40.03,45.0,H-MC,4.69,121.0,-0.13,1.73,3.96,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.20,-17.45,34.20,10.78,67844.0,-41943.0,198375.0,-50.96,12.0,X-LC,6.31,1.0,-0.62,1.41,0.0,X40,NTT,FMCG
7,AWL,485.00,-2.19,-27.77,117.64,57.19,277103.0,-90575.0,235552.0,-65.80,23.0,X-MC,9.28,58.0,-0.33,1.67,0.0,XY24,NTT,FMCG
52,PAGEIND,51605.08,-1.42,-13.26,50.58,30.62,104003.0,-31434.0,205620.0,-38.32,25.0,X-MC,16.43,57.0,-0.30,1.46,0.0,X40,ATH,APPARELS
8,BAJAJHFL,181.50,-1.99,-20.32,95.12,55.47,184394.0,-49432.0,193854.0,-29.32,31.0,X-MC,16.36,63.0,-0.27,1.37,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.47,-40.48,127.15,35.20,97382.0,-52082.0,76588.0,0.36,31.0,X-SC,17.35,93.0,-0.53,0.54,0.0,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-1.22,-2.03,23.27,20.77,53380.0,-4748.0,229396.0,-10.63,31.0,X-MC,0.98,70.0,-0.09,1.63,4.75,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-1.75,-7.23,34.19,24.48,84799.0,-19341.0,248024.0,-8.80,39.0,X-LC,1.70,37.0,-0.23,1.76,43.76,XY24,BTT,FINANCE
65,SIEMENS,4671.50,2.26,-17.32,51.80,25.51,79705.0,-32225.0,153870.0,-5.62,46.0,H-LC,1.72,50.0,-0.40,1.09,13.39,AR,ATH,ELECTRICAL
81,VBL,671.64,-1.73,1.18,34.09,35.67,108943.0,3732.0,319574.0,-10.77,62.0,X-LC,2.19,5.0,0.03,2.27,15.20,X40N,ATH,FMCG
20,DABUR,735.00,0.10,1.66,41.36,43.70,104300.0,4108.0,252176.0,-6.68,65.0,X-MC,2.24,73.0,0.04,1.79,16.96,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.18,-28.43,129.35,64.15,220296.0,-67641.0,170310.0,-58.06,33.0,X-SC,6.29,84.0,-0.31,1.21,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-2.19,-27.77,117.64,57.19,277103.0,-90575.0,235552.0,-65.80,23.0,X-MC,9.28,58.0,-0.33,1.67,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.47,-40.48,127.15,35.20,97382.0,-52082.0,76588.0,0.36,31.0,X-SC,17.35,93.0,-0.53,0.54,0.0,X40,NTT,FOOTWEAR
8,BAJAJHFL,181.50,-1.99,-20.32,95.12,55.47,184394.0,-49432.0,193854.0,-29.32,31.0,X-MC,16.36,63.0,-0.27,1.37,0.0,X40N,ATH,FINANCE
52,PAGEIND,51605.08,-1.42,-13.26,50.58,30.62,104003.0,-31434.0,205620.0,-38.32,25.0,X-MC,16.43,57.0,-0.30,1.46,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-2.31,-28.22,99.01,42.84,46196.0,-18348.0,46658.0,-53.17,49.0,X-SC,27.05,83.0,-0.40,0.33,5.56,XY24,NTT,MISC
58,RELAXO,1176.00,-0.46,-48.45,195.44,52.31,146254.0,-70327.0,74833.0,-45.12,39.0,X-SC,7.83,92.0,-0.48,0.53,0.17,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.47,-40.48,127.15,35.20,97382.0,-52082.0,76588.0,0.36,31.0,X-SC,17.35,93.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.57,-1.30,25.71,24.08,32215.0,-1647.0,125303.0,-9.34,47.0,X-SC,6.29,91.0,-0.05,0.89,18.06,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.96,-17.96,74.70,43.32,99814.0,-29252.0,133620.0,-29.54,42.0,X-SC,7.88,90.0,-0.29,0.95,2.60,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.20,-17.45,34.20,10.78,67844.0,-41943.0,198375.0,-50.96,12.0,X-LC,6.31,1.0,-0.62,1.41,0.00,X40,NTT,FMCG
75,TMPV,600.00,-1.89,-27.72,70.14,22.98,114522.0,-62621.0,163277.0,-25.52,40.0,X-LC,4.16,2.0,-0.55,1.16,2.69,XY24,NTT,AUTO
73,TCS,4389.96,0.12,-11.80,36.85,20.71,122936.0,-44626.0,333611.0,-23.95,47.0,X-LC,2.48,3.0,-0.36,2.37,11.47,X40,ATH,IT
81,VBL,671.64,-1.73,1.18,34.09,35.67,108943.0,3732.0,319574.0,-10.77,62.0,X-LC,2.19,5.0,0.03,2.27,15.20,X40N,ATH,FMCG
41,INFY,1972.00,-0.17,9.64,22.45,34.24,77012.0,30148.0,343036.0,-14.57,49.0,X-LC,7.66,6.0,0.39,2.43,18.72,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.38,-35.03,104.53,32.89,52435.0,-27044.0,50163.0,-719.77,56.0,L-MC,7.70,259.0,-0.52,0.36,38.14,XR,NTT,BANKS
13,BSOFT,831.70,-2.02,-23.32,96.64,50.79,104228.0,-32801.0,107852.0,-1.82,48.0,H-SC,10.62,171.0,-0.31,0.76,25.58,XR,ATH,IT
69,SURYODAY,216.00,-0.11,-17.50,49.14,23.03,72593.0,-31344.0,147727.0,60.09,59.0,H-SC,4.14,165.0,-0.43,1.05,46.57,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-1.54,-14.60,17.12,0.02,25521.0,-25486.0,149073.0,127.12,53.0,M-SC,8.06,240.0,-1.00,1.06,29.67,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-1.61,2.89,15.88,19.23,24584.0,4348.0,154808.0,21.35,65.0,L-LC,7.04,182.0,0.18,1.10,24.78,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.67,-30.04,61.38,12.90,93825.0,-65637.0,152859.0,124.76,61.0,H-SC,10.25,173.0,-0.70,1.08,27.10,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,-0.91,-18.36,61.73,32.03,66438.0,-24209.0,107626.0,-11.67,63.0,H-SC,6.89,157.0,-0.36,0.76,20.55,OX40N,NTT,IT
17,COALINDIA,484.83,-1.61,2.89,15.88,19.23,24584.0,4348.0,154808.0,21.35,65.0,L-LC,7.04,182.0,0.18,1.10,24.78,XY25,ATH,MINING
36,ICICIPRULI,790.00,0.78,13.63,15.05,30.73,33689.0,26843.0,223848.0,-12.89,72.0,X-MC,6.61,79.0,0.80,1.59,28.13,X40,ATH,INSURANCE
82,VOLTAS,1856.00,-0.63,14.21,27.13,45.20,59411.0,27243.0,218985.0,3.07,61.0,X-MC,4.88,80.0,0.46,1.55,21.97,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.40
1,25,44.47
2,50,76.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.69
MC    30.72
LC    26.62
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.36
X40      24.00
X40N     12.82
XY25     10.70
XR        8.96
AR        8.57
OX40N     7.67
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.33
X-LC    23.14
H-SC    22.81
M-SC    11.30
X-SC     7.81
H-MC     4.61
M-MC     1.42
M-LC     1.29
L-LC     1.10
H-LC     1.09
L-SC     0.77
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.13,-9.06,44.84
IT,12.87,-17.66,77.00
FINANCE,11.08,-14.04,61.90
MISC,7.04,-30.78,84.27
ELECTRICAL,5.96,-11.70,52.61
PAINTS,5.78,-13.18,30.00
INSURANCE,4.74,-1.55,36.30
PHARMA,3.94,-5.85,39.17
AUTO,2.78,-33.89,80.31


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3253400.0,21
AR,1362378.0,10
XR,1230504.0,12
X40,1136958.0,15
X40N,933877.0,9
OX40N,709093.0,10
XY25,433880.0,7
SR,286378.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3573740.0,24
X-MC,1601949.0,16
M-SC,1467301.0,15
X-LC,963102.0,13
X-SC,817350.0,8
H-MC,428838.0,3
L-SC,182325.0,2
M-LC,110020.0,1
H-LC,79705.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1255517.0      6
           AR         927591.0      5
M-SC       XY24       845295.0      6
H-SC       XR         794051.0      7
X-MC       X40        517532.0      7
           XY24       449361.0      3
X-LC       X40        421675.0      6
X-MC       X40N       407865.0      4
X-SC       X40N       320892.0      3
H-SC       OX40N      310203.0      4
M-SC       OX40N      305902.0      5
X-SC       XY24       298707.0      3
H-SC       SR         286378.0      2
X-MC       XY25       227191.0      2
H-MC       AR         223639.0      2
           XY24       205199.0      1
X-LC       X40N       205120.0      2
           XY24       199321.0      2
X-SC       X40        197751.0      2
M-SC       XR         184661.0      2
X-LC       XY25       136986.0      3
M-SC       AR         131443.0      2
M-LC       XR         110020.0      1
L-SC       OX40N       92988.0      1
           XR          89337.0      1
H-LC       AR          79705.0      1
L-MC       XR          52435.0      1
M-MC       XY25        45119.0      1
L-LC       XY25        24584.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
